In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
from urllib import request
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
url = "https://www.jumia.co.ke/all-products/"
requests = requests.get(url)
## Parsing the html soup
soup = BeautifulSoup(requests.text,"html.parser")
items = soup.find_all("article", attrs = {"class":"prd _fb col c-prd"})

In [3]:
print(len(items))

40


In [4]:
print(items[0])

<article class="prd _fb col c-prd"><a class="btn _i _rnd -mas -fsh0 -me-start _wslt _sec" data-pop-open="addToWishlist" data-pop-trig="atw" data-sku="CA733DR0DOEJ5NAFAMZ" rel="nofollow"><svg aria-label="Add to wishlist" class="ic -f-or5" height="16" viewbox="0 0 24 24" width="16"><use xlink:href="https://www.jumia.co.ke/assets_he/images/i-icons.a66628fd.svg#saved-items"></use></svg></a><a class="core" data-gtm-brand="Campo Viejo" data-gtm-category="Grocery/Drinks/Beer, Wine &amp; Spirits/Wine/Red Wine" data-gtm-dimension23="" data-gtm-dimension26="9" data-gtm-dimension27="4.7" data-gtm-dimension28="1" data-gtm-dimension37="0" data-gtm-dimension43="BLF|Boost|COL_226|COL_240|COL_262|COL_307|COL_323|CP_11|CP_23|CP_24|CP_26|CP_MT102|CP_MT104|CP_MT133|CP_MT134|CP_UN14|DOW_100|JMALL|TBOOST|WCS" data-gtm-dimension44="0" data-gtm-id="CA733DR0DOEJ5NAFAMZ" data-gtm-list="" data-gtm-name="Campo Viejo Reserva Red Wine - 750 ML" data-gtm-position="1" data-gtm-price="10.44" data-track-onclick="eecPr

In [5]:
first_items = items[0]
first_items_name =first_items.find("h3",attrs = {"class":"name"}).text
print(first_items_name)

Campo Viejo Campo Viejo Reserva Red Wine - 750 ML


In [6]:
##discount
first_items = items[0]
first_items_discount = first_items.find("div", attrs = {"class":"bdg _dsct _sm"}).text
print(first_items_discount)

17%


In [7]:
records= []
for item in items:
    try:
        item_name =item.find("h3",attrs = {"class":"name"}).text
        current_price =item.find("div", attrs = {"class":"prc"}).text
        old_price_element = item.find("div", attrs = {"class":"old"})
        item_discount_element = item.find("div", attrs = {"class":"bdg _dsct _sm"})
        item_link = item.find("a", attrs = {"class":"core"}).get("href")

        if old_price_element: 
            old_price = old_price_element.text
        else:
            old_price = None
        if item_discount_element:
            item_discount = item_discount_element.text
        else:
            item_dicount = None
        records.append(( item_name, current_price,old_price, item_discount, item_link))

    except AttributeError:
        records.append(None)
print(len(records))


40


In [8]:
import time
import requests
from bs4 import BeautifulSoup

base_url = "https://www.jumia.co.ke/all-products/?page={}#catalog-listing"
records = []

for page_number in range(1, 51):
    url = base_url.format(page_number)
    time.sleep(5)
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        print(f"Page {page_number} scrapped successfully")
        items = soup.find_all("article", attrs={"class": "prd _fb col c-prd"})
        for item in items:
            try:
                item_name = item.find("h3", attrs={"class": "name"}).text
                current_price = item.find("div", attrs={"class": "prc"}).text
                old_price_element = item.find("div", attrs={"class": "old"})
                item_discount_element = item.find("div", attrs={"class": "bdg _dsct _sm"})
                item_link = item.find("a", attrs={"class": "core"}).get("href")
                
                if old_price_element:
                    old_price = old_price_element.text
                else:
                    old_price = None
                if item_discount_element:
                    item_discount = item_discount_element.text
                else:
                    item_discount = None
                records.append((item_name, current_price, old_price, item_discount, item_link))
            except AttributeError:
                records.append(None)
    else:
        print(f"Page {page_number} not scrapped")

Page 1 scrapped successfully
Page 2 scrapped successfully
Page 3 scrapped successfully
Page 4 scrapped successfully
Page 5 scrapped successfully
Page 6 scrapped successfully
Page 7 scrapped successfully
Page 8 scrapped successfully
Page 9 scrapped successfully
Page 10 scrapped successfully
Page 11 scrapped successfully
Page 12 scrapped successfully
Page 13 scrapped successfully
Page 14 scrapped successfully
Page 15 scrapped successfully
Page 16 scrapped successfully
Page 17 scrapped successfully
Page 18 scrapped successfully
Page 19 scrapped successfully
Page 20 scrapped successfully
Page 21 scrapped successfully
Page 22 scrapped successfully
Page 23 scrapped successfully
Page 24 scrapped successfully
Page 25 scrapped successfully
Page 26 scrapped successfully
Page 27 scrapped successfully
Page 28 scrapped successfully
Page 29 scrapped successfully
Page 30 scrapped successfully
Page 31 scrapped successfully
Page 32 scrapped successfully
Page 33 scrapped successfully
Page 34 scrapped su

In [9]:
# Save records to CSV file
import csv
csv_file = "jumia_products.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Item Name", "Current Price", "Old Price", "Discount", "Item Link"])
    writer.writerows(records)

print("Data saved to", csv_file)

Data saved to jumia_products.csv


In [10]:
df = pd.read_csv("jumia_products.csv")

In [11]:
print("\nSummary statistics:")
df.describe().T


Summary statistics:


,count,unique,top,freq
Item Name,2000,1840,Motorola Moto Edge Plus Leather Phone Case,5
Current Price,2000,1112,KSh 799,30
Old Price,1744,904,"KSh 1,500",48
Discount,1744,63,50%,270
Item Link,2000,1880,/fashion-winner-mechanical-watch-for-men-skele...,2


In [12]:
print("\nMissing values:")
df.isnull().sum()


Missing values:


Item Name          0
Current Price      0
Old Price        256
Discount         256
Item Link          0
dtype: int64